In [1]:
import sys
sys.path.append("./")

In [7]:
def predict(x_i, beta):
    return dot(x_i, beta)

In [8]:
def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

In [9]:
def squared_error(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

In [10]:
def squared_error_gradient(x_i, y_i, beta):
    return [-2 * x_ij * error(x_i, y_i, beta) for x_ij in x_i]

In [5]:
from lib.lin_alg_gradient_descent import *
def estimate_beta(x, y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error,
                              squared_error_gradient,
                              x,
                              y,
                              beta_initial,
                              0.001)

In [2]:

def multiple_r_squared(x, y, beta):
    sum_of_squared_errors = sum(error(x_i, y_i, beta) ** 2
                                for x_i, y_i in zip(x, y))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(y)

def bootstrap_sample(data):
    """randomly samples len(data) elements with replacement"""
    return [random.choice(data) for _ in data]

def bootstrap_statistic(data, stats_fn, num_samples):
    """evaluates stats_fn on num_samples bootstrap samples from data"""
    return [stats_fn(bootstrap_sample(data))
            for _ in range(num_samples)]

def estimate_sample_beta(sample):
    x_sample, y_sample = list(zip(*sample)) # magic unzipping trick
    return estimate_beta(x_sample, y_sample)

def p_value(beta_hat_j, sigma_hat_j):
    if beta_hat_j > 0:
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)

#
# REGULARIZED REGRESSION
#

# alpha is a *hyperparameter* controlling how harsh the penalty is
# sometimes it's called "lambda" but that already means something in Python
def ridge_penalty(beta, alpha):
  return alpha * dot(beta[1:], beta[1:])

def squared_error_ridge(x_i, y_i, beta, alpha):
    """estimate error plus ridge penalty on beta"""
    return error(x_i, y_i, beta) ** 2 + ridge_penalty(beta, alpha)

def ridge_penalty_gradient(beta, alpha):
    """gradient of just the ridge penalty"""
    return [0] + [2 * alpha * beta_j for beta_j in beta[1:]]

def squared_error_ridge_gradient(x_i, y_i, beta, alpha):
    """the gradient corresponding to the ith squared error term
    including the ridge penalty"""
    return vector_add(squared_error_gradient(x_i, y_i, beta),
                      ridge_penalty_gradient(beta, alpha))

def estimate_beta_ridge(x, y, alpha):
    """use gradient descent to fit a ridge regression
    with penalty alpha"""
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(partial(squared_error_ridge, alpha=alpha),
                               partial(squared_error_ridge_gradient,
                                       alpha=alpha),
                               x, y,
                               beta_initial,
                               0.001)

def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])

In [11]:
import random

x = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

random.seed(0)
beta = estimate_beta(x, daily_minutes_good) # [30.63, 0.972, -1.868, 0.911]
print("beta", beta)
print("r-squared", multiple_r_squared(x, daily_minutes_good, beta))
print()

print("digression: the bootstrap")
# 101 points all very close to 100
close_to_100 = [99.5 + random.random() for _ in range(101)]

# 101 points, 50 of them near 0, 50 of them near 200
far_from_100 = ([99.5 + random.random()] +
                [random.random() for _ in range(50)] +
                [200 + random.random() for _ in range(50)])

print("bootstrap_statistic(close_to_100, median, 100):")
print(bootstrap_statistic(close_to_100, median, 100))
print("bootstrap_statistic(far_from_100, median, 100):")
print(bootstrap_statistic(far_from_100, median, 100))
print()

random.seed(0) # so that you get the same results as me

bootstrap_betas = bootstrap_statistic(list(zip(x, daily_minutes_good)),
                                      estimate_sample_beta,
                                      100)

bootstrap_standard_errors = [
    standard_deviation([beta[i] for beta in bootstrap_betas])
    for i in range(4)]

print("bootstrap standard errors", bootstrap_standard_errors)
print()

print("p_value(30.63, 1.174)", p_value(30.63, 1.174))
print("p_value(0.972, 0.079)", p_value(0.972, 0.079))
print("p_value(-1.868, 0.131)", p_value(-1.868, 0.131))
print("p_value(0.911, 0.990)", p_value(0.911, 0.990))
print()

print("regularization")

random.seed(0)
for alpha in [0.0, 0.01, 0.1, 1, 10]:
    beta = estimate_beta_ridge(x, daily_minutes_good, alpha=alpha)
    print("alpha", alpha)
    print("beta", beta)
    print("dot(beta[1:],beta[1:])", dot(beta[1:], beta[1:]))
    print("r-squared", multiple_r_squared(x, daily_minutes_good, beta))
print()


	 min_theta, min_value [0.8444218515250481, 0.7579544029403025, 0.420571580830845, 0.25891675029296335] 106652.57996908313
		 theta [3.7756090874697907, 2.8790059086447664, 0.9047221818349325, 2.4331996690468554]
	 min_theta, min_value [3.7756090874697907, 2.8790059086447664, 0.9047221818349325, 2.4331996690468554] 49093.80207574332
		 theta [6.028469434362678, 1.2176295527752414, 1.1325935593485976, 4.231153843452433]
	 min_theta, min_value [6.028469434362678, 1.2176295527752414, 1.1325935593485976, 4.231153843452433] 35371.85925093048
		 theta [7.89488342500121, -2.264330409667861, -0.874373073104022, 5.571741877219619]
	 alpha 0.0009000000000000001
		 theta [9.796122849948327, 2.4005801797776183, 0.3526628699379181, 6.402877521374613]
	 min_theta, min_value [9.796122849948327, 2.4005801797776183, 0.3526628699379181, 6.402877521374613] 28883.044982344836
		 theta [10.992242820810281, 2.34203532061706, -0.4820174109961292, 7.140999036649148]
	 min_theta, min_value [10.992242820810281,

NameError: name 'total_sum_of_squares' is not defined

In [16]:
import random
from lib.lin_alg_working_with_data import *

test_data = csv_read("data/loanf.csv", [int, float, int, int, float, int])
test_data = [row for row in test_data if row[0] != None]
print(test_data[:10])

interest_rate = get_column(test_data, 1)
fifo_score = get_column(test_data, 2)
loan_amount = get_column(test_data, 5)
X = list(zip(fifo_score, interest_rate))
Y = loan_amount


random.seed(0)
beta = estimate_beta(X, Y)
print(beta)

[[6, 15.31, 670, 36, 4891.67, 6000], [11, 19.72, 670, 36, 3575.0, 2000], [12, 14.27, 665, 36, 4250.0, 10625], [13, 21.67, 670, 60, 14166.67, 28000], [21, 21.98, 665, 36, 6666.67, 22000], [23, 17.99, 665, 60, 4100.0, 6500], [25, 16.82, 670, 60, 4500.0, 4000], [40, 12.49, 660, 36, 3448.33, 2400], [41, 17.27, 670, 36, 2500.0, 3200], [43, 19.13, 660, 36, 12000.0, 5500]]
	 min_theta, min_value [0.8444218515250481, 0.7579544029403025] 500096293107.3055
		 theta [nan, nan]
	 alpha 0.0009000000000000001
		 theta [nan, nan]
	 alpha 0.0008100000000000001
		 theta [nan, nan]
	 alpha 0.000729
		 theta [nan, nan]
	 alpha 0.0006561000000000001
		 theta [nan, nan]
	 alpha 0.00059049
		 theta [nan, nan]
	 alpha 0.000531441
		 theta [nan, nan]
	 alpha 0.0004782969
		 theta [nan, nan]
	 alpha 0.00043046721
		 theta [nan, nan]
	 alpha 0.000387420489
		 theta [nan, nan]
	 alpha 0.0003486784401
		 theta [nan, nan]
	 alpha 0.00031381059609000004
		 theta [nan, nan]
	 alpha 0.00028242953648100003
		 theta [n

In [ ]:
import matplotlib.pyplot as plt
